In [8]:
import librosa
import numpy as np
import os
import math
from sklearn.cluster import KMeans
import hmmlearn.hmm
import noisereduce as nr
import sounddevice as sd
import soundfile as sf
import pickle

In [9]:
# def noise_cancel(filename, classs, dirr):
#     data, fs = librosa.load(os.path.join(dirr,filename))
#     reduced_noise = nr.reduce_noise(audio_clip=data, noise_clip=data)
#     sf.write('./normal_word/' + classs + '/' + filename, data=reduced_noise, samplerate=fs)
# def normal_data():
#     class_names = ["nguoi", "toi", "khong", "mot" , "test_khong", "benh_nhan" , "test_toi" , "test_mot", "test_nguoi", "test_benh_nhan" ]
#     for cname in class_names:
#         files = os.listdir(os.path.join("word", cname))
#         dirfile = os.path.join("word", cname)
#         for f in files:
#             if f.endswith(".wav"):
#                 noise_cancel(f, cname, dirfile)
# normal_data()

In [10]:

def get_mfcc(file_path):
    y, sr = librosa.load(file_path) # read .wav file
    hop_length = math.floor(sr*0.010) # 10ms hop
    win_length = math.floor(sr*0.025) # 25ms frame
    # mfcc is 12 x T matrix
    mfcc = librosa.feature.mfcc(
        y, sr, n_mfcc=12, n_fft=1024,
        hop_length=hop_length, win_length=win_length)
    # substract mean from mfcc --> normalize mfcc
    mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 
    # delta feature 1st order and 2nd order
    delta1 = librosa.feature.delta(mfcc, order=1)
    delta2 = librosa.feature.delta(mfcc, order=2)
    # X is 36 x T
    X = np.concatenate([mfcc, delta1, delta2], axis=0) # O^r
    # return T x 36 (transpose of X)
    return X.T # hmmlearn use T x N matrix

def get_class_data(data_dir):
    files = os.listdir(data_dir)
    mfcc = [get_mfcc(os.path.join(data_dir,f)) for f in files if f.endswith(".wav")]
    print('data_dir: ', data_dir)
    print(f'mfcc.shape: {np.array(mfcc).shape}')
    return mfcc

def clustering(X, n_clusters=20):
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0, verbose=0)
    kmeans.fit(X)
    print("centers", kmeans.cluster_centers_.shape)
    return kmeans  


In [11]:

class_names = ["nguoi", "toi", "khong", "mot", "benh_nhan" , "test_khong" , "test_toi" , "test_mot", "test_nguoi", "test_benh_nhan" ]
dataset = {}
for cname in class_names:
    print(f"Load {cname} dataset")
    dataset[cname] = get_class_data(os.path.join("word", cname))


Load nguoi dataset
data_dir:  word/nguoi
mfcc.shape: (76,)
Load toi dataset
data_dir:  word/toi
mfcc.shape: (76,)
Load khong dataset
data_dir:  word/khong
mfcc.shape: (76,)
Load mot dataset
data_dir:  word/mot
mfcc.shape: (76,)
Load benh_nhan dataset
data_dir:  word/benh_nhan
mfcc.shape: (75,)
Load test_khong dataset
data_dir:  word/test_khong
mfcc.shape: (25,)
Load test_toi dataset
data_dir:  word/test_toi
mfcc.shape: (25,)
Load test_mot dataset
data_dir:  word/test_mot
mfcc.shape: (25,)
Load test_nguoi dataset
data_dir:  word/test_nguoi
mfcc.shape: (25,)
Load test_benh_nhan dataset
data_dir:  word/test_benh_nhan
mfcc.shape: (25,)


In [12]:
# Get all vectors in the datasets
all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in dataset.items()], axis=0)
print("vectors", all_vectors.shape)

vectors (14083, 36)


In [13]:
for k, v in dataset.items():
    print(v)

[array([[ 5.34689636e+01,  1.61920319e+01,  3.38034821e+01,
        -3.76386528e+01,  5.51878262e+00,  3.48067322e+01,
         3.37554665e+01,  4.84276199e+00,  2.78776665e+01,
         1.48108521e+01,  5.92070007e+00, -7.19398499e+00,
         1.11748248e-01, -1.35886419e+00, -3.83443332e+00,
         8.80451870e+00, -3.99910480e-01, -8.04342079e+00,
        -4.31577158e+00,  1.48417222e+00, -3.61021256e+00,
        -1.29039860e+00, -6.58610523e-01,  3.37447500e+00,
        -2.93800569e+00,  1.00206113e+00,  4.40491629e+00,
         1.63842523e+00,  1.89591420e+00,  1.41090441e+00,
         3.45066977e+00,  4.02468157e+00, -4.45886523e-01,
         2.70042330e-01,  1.58663881e+00,  2.07727838e+00],
       [ 5.11544800e+01, -2.06883240e+00, -5.84616470e+00,
        -4.82370834e+01, -1.51896458e+01,  3.24105530e+01,
         1.61914177e+01, -2.22229652e+01,  1.31120911e+01,
         5.32440758e+00,  7.59771156e+00, -1.57441998e+01,
         1.11748248e-01, -1.35886419e+00, -3.83443332e

[array([[ 8.26626587e+00, -3.29721146e+01, -2.67021866e+01,
        -1.14926805e+01, -2.00656052e+01, -6.85297441e+00,
        -8.83157349e+00,  8.10877895e+00,  2.88952017e+00,
        -1.51307278e+01,  9.11798954e+00, -4.74039888e+00,
         1.47310126e+00,  7.24396753e+00,  3.00941181e+00,
        -9.84182179e-01,  1.09414089e+00, -1.62289131e+00,
         1.17695606e+00, -2.55906248e+00, -5.42390347e-01,
         1.47991085e+00, -3.50787997e+00,  1.72490254e-01,
        -1.81471765e+00, -5.43490171e+00, -1.78709316e+00,
        -3.72601837e-01,  1.32081199e+00,  1.17444253e+00,
        -3.02785516e-01,  1.38322580e+00,  1.66698325e+00,
         4.24880028e-01, -5.99417984e-02, -6.24355555e-01],
       [ 1.79446411e+01, -2.18443146e+01, -2.46011696e+01,
        -3.76716805e+00, -1.20767384e+01,  1.42351770e+01,
        -9.58531475e+00,  6.88158464e+00,  6.47027302e+00,
         1.02459278e+01,  2.55063896e+01,  9.97895241e-01,
         1.47310126e+00,  7.24396753e+00,  3.00941181e

[array([[ 8.89428711e+01, -1.06353745e+02, -2.38827801e+01,
        -3.57728233e+01,  1.71853790e+01,  8.33695412e+00,
        -1.27247047e+01, -1.15151129e+01,  1.66891232e+01,
         3.51655579e+01,  1.73445778e+01,  2.52989655e+01,
        -5.75099039e+00,  2.21760139e+01,  2.30689478e+00,
         2.69809866e+00, -2.78404212e+00, -3.88434529e+00,
        -1.46881068e+00, -1.02359462e+00, -3.13928843e+00,
        -5.97987986e+00, -5.25243187e+00, -6.09513807e+00,
         6.71545327e-01,  5.29352665e+00, -8.01209271e-01,
         4.83480573e-01,  7.89816618e-01, -2.31460738e+00,
        -1.06117487e+00, -7.94468462e-01, -2.87558705e-01,
        -1.59674242e-01, -5.86229712e-02, -1.02451718e+00],
       [ 8.86329956e+01, -1.02103470e+02, -3.46856461e+01,
        -4.14613762e+01, -1.20362129e+01,  2.25290337e+01,
        -6.99340534e+00, -7.41856277e-01,  1.00802345e+01,
         3.37401619e+01,  1.58655815e+01,  3.08298702e+01,
        -5.75099039e+00,  2.21760139e+01,  2.30689478e

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
# Run K-Means algorithm to get clusters
kmeans = clustering(all_vectors, n_clusters=20)
pickle.dump(kmeans, open('./model/kmean.pk','wb'))
print("centers", kmeans.cluster_centers_.shape)


centers (20, 36)
centers (20, 36)


In [17]:
for key, val in dataset.items():
    print(key,'\n', np.array(val).shape)


nguoi 
 (76,)
toi 
 (76,)
khong 
 (76,)
mot 
 (76,)
benh_nhan 
 (75,)
test_khong 
 (25,)
test_toi 
 (25,)
test_mot 
 (25,)
test_nguoi 
 (25,)
test_benh_nhan 
 (25,)


In [18]:
models = {}
original_dataset = {}

In [19]:
original_dataset['nguoi'] = dataset['nguoi'].copy()

In [20]:
cname = 'nguoi'
# convert all vectors to the cluster index
# dataset['one'] = [O^1, ... O^R] , O^r: the r-th recorded wav file 
# O^r = (c1, c2, ... ct, ... cT) , c_i: the i-th frame in the r-th observation ( or the r-th wav file )
# O^r size T x 1
dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in original_dataset[cname]])


hmm = hmmlearn.hmm.MultinomialHMM(
    n_components=9, random_state=0, n_iter=1000, verbose=True,
    params='te',
    init_params='e'
)
hmm.startprob_=np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
hmm.transmat_=np.array([
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
    
])
if cname[:4] != 'test':
    X = np.concatenate(dataset[cname])
    lengths = list([len(x) for x in dataset[cname]])
    print("training class", cname)
    print(X.shape, lengths, len(lengths))
    hmm.fit(X, lengths=lengths)
    models[cname] = hmm

training class nguoi
(1875, 1) [16, 17, 17, 28, 27, 22, 23, 20, 21, 25, 12, 13, 26, 22, 44, 22, 17, 14, 21, 22, 27, 30, 22, 20, 21, 18, 29, 37, 14, 23, 23, 19, 23, 18, 37, 15, 24, 23, 17, 31, 35, 40, 20, 30, 40, 20, 44, 28, 19, 27, 22, 19, 16, 23, 25, 20, 42, 19, 23, 29, 30, 21, 19, 28, 31, 21, 19, 27, 28, 22, 20, 33, 31, 30, 24, 50] 76


         1       -5329.0062             +nan
         2       -3869.3663       +1459.6399
         3       -3473.9696        +395.3967
         4       -3217.0511        +256.9185
         5       -3072.6097        +144.4414
         6       -2973.1796         +99.4301
         7       -2925.1596         +48.0200
         8       -2888.9187         +36.2409
         9       -2848.7602         +40.1585
        10       -2810.0129         +38.7473
        11       -2785.4026         +24.6103
        12       -2773.3108         +12.0918
        13       -2764.8896          +8.4212
        14       -2756.7634          +8.1262
        15       -2752.0936          +4.6697
        16       -2749.8967          +2.1969
        17       -2748.7430          +1.1538
        18       -2748.1158          +0.6272
        19       -2747.7611          +0.3546
        20       -2747.5478          +0.2134
        21       -2747.4097          +0.1380
        22       -2747.3137          +0.0960
        23

In [21]:
original_dataset['toi'] = dataset['toi'].copy()
cname = 'toi'
class_vectors = dataset[cname]
# convert all vectors to the cluster index
# dataset['one'] = [O^1, ... O^R] , O^r: the r-th recorded wav file 
# O^r = (c1, c2, ... ct, ... cT) , c_i: the i-th frame in the r-th observation ( or the r-th wav file )
# O^r size T x 1

dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in original_dataset[cname]])


hmm = hmmlearn.hmm.MultinomialHMM(
    n_components= 9, random_state=0, n_iter=1000, verbose=True,
    params='te',
    init_params='e'
)
hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
hmm.transmat_ = np.array([ 
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
    ])

if cname[:4] != 'test':
    X = np.concatenate(dataset[cname])
    lengths = list([len(x) for x in dataset[cname]])
    print("training class", cname)
    print(X.shape, lengths, len(lengths))
    hmm.fit(X, lengths=lengths)
    models[cname] = hmm

         1       -4948.5071             +nan
         2       -3699.5483       +1248.9589


training class toi
(1736, 1) [17, 16, 18, 24, 10, 21, 14, 17, 28, 25, 16, 16, 25, 33, 15, 24, 37, 12, 27, 23, 27, 17, 16, 35, 20, 15, 14, 38, 32, 37, 33, 43, 34, 17, 29, 19, 20, 20, 17, 23, 15, 22, 17, 15, 42, 33, 27, 27, 26, 32, 14, 18, 32, 16, 14, 15, 30, 18, 23, 41, 13, 16, 17, 20, 14, 12, 47, 25, 21, 27, 15, 24, 14, 39, 13, 18] 76


         3       -3329.0754        +370.4729
         4       -3152.0851        +176.9903
         5       -3082.0092         +70.0759
         6       -3039.4643         +42.5450
         7       -3016.4149         +23.0494
         8       -3003.0770         +13.3379
         9       -2991.6207         +11.4563
        10       -2984.3292          +7.2914
        11       -2968.9777         +15.3515
        12       -2942.1316         +26.8461
        13       -2930.2117         +11.9199
        14       -2906.1887         +24.0231
        15       -2888.6613         +17.5274
        16       -2881.6312          +7.0301
        17       -2874.0188          +7.6124
        18       -2868.2353          +5.7834
        19       -2864.8058          +3.4295
        20       -2862.7937          +2.0121
        21       -2861.5884          +1.2053
        22       -2860.7387          +0.8497
        23       -2859.9652          +0.7736
        24       -2859.2113          +0.7539
        25

In [22]:
np.set_printoptions(precision=2, suppress=True)
print(models['toi'].transmat_)

[[0.67 0.11 0.22 0.   0.   0.   0.   0.   0.  ]
 [0.   0.88 0.05 0.06 0.   0.   0.   0.   0.  ]
 [0.   0.   0.64 0.07 0.29 0.   0.   0.   0.  ]
 [0.   0.   0.   0.78 0.04 0.17 0.   0.   0.  ]
 [0.   0.   0.   0.   0.83 0.16 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.73 0.26 0.01 0.  ]
 [0.   0.   0.   0.   0.   0.   0.86 0.   0.14]
 [0.   0.   0.   0.   0.   0.   0.   0.97 0.03]
 [0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]


In [23]:
original_dataset['khong'] = dataset['khong'].copy()
cname = 'khong'
class_vectors = dataset[cname]
# convert all vectors to the cluster index
# dataset['one'] = [O^1, ... O^R] , O^r: the r-th recorded wav file 
# O^r = (c1, c2, ... ct, ... cT) , c_i: the i-th frame in the r-th observation ( or the r-th wav file )
# O^r size T x 1
dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])


hmm = hmmlearn.hmm.MultinomialHMM(
    n_components=9, random_state=0, n_iter=1000, verbose=True,
    startprob_prior=np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    transmat_prior=np.array([ 
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
    ])
)

if cname[:4] != 'test':
    X = np.concatenate(dataset[cname])
    lengths = list([len(x) for x in dataset[cname]])
    print("training class", cname)
    print(X.shape, lengths, len(lengths))
    hmm.fit(X, lengths=lengths)
    models[cname] = hmm

         1       -5902.1990             +nan


training class khong
(1898, 1) [27, 17, 27, 27, 33, 21, 20, 23, 29, 45, 25, 26, 27, 33, 35, 20, 30, 20, 28, 16, 22, 21, 15, 28, 24, 17, 22, 24, 18, 26, 22, 23, 25, 33, 25, 14, 33, 21, 20, 27, 23, 19, 20, 22, 25, 19, 30, 31, 18, 20, 21, 19, 20, 28, 25, 17, 26, 20, 38, 40, 25, 27, 25, 34, 34, 20, 24, 23, 23, 28, 27, 25, 26, 36, 32, 19] 76


         2       -4826.0719       +1076.1271
         3       -4775.2307         +50.8412
         4       -4653.9753        +121.2554
         5       -4397.5409        +256.4344
         6       -4027.0067        +370.5342
         7       -3634.2410        +392.7657
         8       -3425.9351        +208.3059
         9       -3335.6783         +90.2568
        10       -3254.1136         +81.5647
        11       -3188.7314         +65.3822
        12       -3139.5402         +49.1912
        13       -3104.3065         +35.2337
        14       -3079.8726         +24.4339
        15       -3061.8558         +18.0168
        16       -3048.9135         +12.9424
        17       -3041.9626          +6.9508
        18       -3036.4207          +5.5419
        19       -3031.3619          +5.0588
        20       -3025.0712          +6.2907
        21       -3015.1721          +9.8991
        22       -2998.6803         +16.4918
        23       -2979.1060         +19.5743
        24

In [24]:
original_dataset['mot'] = dataset['mot'].copy()
cname = 'mot'
# convert all vectors to the cluster index
# dataset['one'] = [O^1, ... O^R] , O^r: the r-th recorded wav file 
# O^r = (c1, c2, ... ct, ... cT) , c_i: the i-th frame in the r-th observation ( or the r-th wav file )
# O^r size T x 1
dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in original_dataset[cname]])


hmm = hmmlearn.hmm.MultinomialHMM(
    n_components=9, random_state=0, n_iter=1000, verbose=True,
    params='te',
    init_params='e'
)
hmm.startprob_=np.array([0.7,0.2,0.1,0.0,0.0,0.0,0.0,0.0,0.0])
hmm.transmat_=np.array([ 
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
    ])
if cname[:4] != 'test':
    X = np.concatenate(dataset[cname])
    lengths = list([len(x) for x in dataset[cname]])
    print("training class", cname)
    print(X.shape, lengths, len(lengths))
    hmm.fit(X, lengths=lengths)
    models[cname] = hmm

         1       -4302.1104             +nan
         2       -3272.0545       +1030.0559
         3       -3041.7642        +230.2902


training class mot
(1472, 1) [22, 23, 18, 11, 24, 20, 18, 18, 18, 26, 24, 16, 36, 16, 25, 19, 21, 17, 20, 20, 16, 20, 27, 19, 23, 23, 20, 12, 14, 18, 10, 18, 22, 21, 36, 16, 23, 23, 16, 18, 22, 26, 18, 12, 16, 19, 15, 15, 12, 18, 11, 20, 24, 14, 18, 18, 20, 14, 23, 22, 15, 20, 23, 17, 17, 15, 16, 21, 23, 20, 21, 21, 24, 16, 23, 16] 76


         4       -2902.4373        +139.3269
         5       -2805.7942         +96.6431
         6       -2757.2727         +48.5216
         7       -2734.5384         +22.7343
         8       -2714.9673         +19.5711
         9       -2686.7212         +28.2461
        10       -2671.4643         +15.2569
        11       -2664.1295          +7.3348
        12       -2658.7198          +5.4097
        13       -2654.6131          +4.1067
        14       -2651.8362          +2.7770
        15       -2649.6904          +2.1458
        16       -2647.3443          +2.3461
        17       -2644.8129          +2.5314
        18       -2642.8922          +1.9206
        19       -2641.5004          +1.3918
        20       -2640.4198          +1.0807
        21       -2639.5125          +0.9073
        22       -2638.7315          +0.7809
        23       -2638.0699          +0.6617
        24       -2637.5289          +0.5409
        25       -2637.1056          +0.4233
        26

In [25]:
original_dataset['benh_nhan'] = dataset['benh_nhan'].copy()
cname = 'benh_nhan'
# convert all vectors to the cluster index
# dataset['one'] = [O^1, ... O^R] , O^r: the r-th recorded wav file 
# O^r = (c1, c2, ... ct, ... cT) , c_i: the i-th frame in the r-th observation ( or the r-th wav file )
# O^r size T x 1
dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in original_dataset[cname]])


hmm = hmmlearn.hmm.MultinomialHMM(
    n_components=18, random_state=0, n_iter=1000, verbose=True,
    params='te',
    init_params='e'
)
hmm.startprob_=np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],)
hmm.transmat_=np.array([ 
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
    ])
if cname[:4] != 'test':
    X = np.concatenate(dataset[cname])
    lengths = list([len(x) for x in dataset[cname]])
    print("training class", cname)
    print(X.shape, lengths, len(lengths))
    hmm.fit(X, lengths=lengths)
    models[cname] = hmm

         1       -9899.2217             +nan


training class benh_nhan
(3422, 1) [45, 43, 33, 47, 54, 37, 29, 47, 42, 67, 55, 57, 48, 39, 46, 34, 45, 44, 39, 38, 41, 46, 28, 51, 70, 42, 71, 48, 42, 59, 39, 34, 35, 67, 58, 39, 39, 46, 85, 30, 55, 34, 42, 44, 50, 26, 58, 44, 33, 44, 43, 41, 37, 37, 40, 48, 56, 35, 37, 36, 42, 48, 69, 52, 39, 25, 36, 51, 56, 74, 32, 63, 55, 54, 27] 75


         2       -7804.5327       +2094.6890
         3       -6653.6649       +1150.8678
         4       -6048.1424        +605.5224
         5       -5863.2625        +184.8800
         6       -5784.9092         +78.3533
         7       -5731.2099         +53.6993
         8       -5688.0741         +43.1358
         9       -5655.3575         +32.7165
        10       -5637.6650         +17.6925
        11       -5625.6630         +12.0020
        12       -5614.1681         +11.4949
        13       -5605.6959          +8.4722
        14       -5598.0432          +7.6527
        15       -5587.6641         +10.3792
        16       -5575.3347         +12.3293
        17       -5565.6675          +9.6672
        18       -5557.2138          +8.4537
        19       -5548.3458          +8.8680
        20       -5544.5528          +3.7929
        21       -5542.4171          +2.1357
        22       -5538.8637          +3.5534
        23       -5532.8687          +5.9950
        24

In [26]:
pickle.dump(models, open('./model/models.pk','wb'))

In [27]:
dataset["test_nguoi"] = list([kmeans.predict(v).reshape(-1,1) for v in dataset['test_nguoi']])
dataset['test_toi'] = list([kmeans.predict(v).reshape(-1,1) for v in dataset['test_toi']])
dataset['test_khong'] = list([kmeans.predict(v).reshape(-1,1) for v in dataset['test_khong']])
dataset['test_mot'] = list([kmeans.predict(v).reshape(-1,1) for v in dataset['test_mot']])
dataset['test_benh_nhan'] = list([kmeans.predict(v).reshape(-1,1) for v in dataset['test_benh_nhan']])

In [29]:
print("Testing")
mapping = ["nguoi", "toi", "khong", "mot", "benh_nhan"]
class_names = ["test_nguoi", "test_toi", "test_khong", "test_mot", "test_benh_nhan"]
count = 0
correct = 0
for true_cname in class_names:
    print(true_cname)
    score = []
    count = 0
    correct = 0
    for i in dataset[true_cname]:
        score = [model.score(i, [len(i)]) for cname, model in models.items() if cname[:4] != 'test']
        res = mapping[score.index(max(score))] 
        print(res + '-' + true_cname)
        if res == true_cname[5:]:
            correct += 1
        count += 1 
    print(correct/count)

Testing
test_nguoi
nguoi-test_nguoi
nguoi-test_nguoi
nguoi-test_nguoi
nguoi-test_nguoi
nguoi-test_nguoi
nguoi-test_nguoi
benh_nhan-test_nguoi
nguoi-test_nguoi
benh_nhan-test_nguoi
nguoi-test_nguoi
nguoi-test_nguoi
nguoi-test_nguoi
nguoi-test_nguoi
benh_nhan-test_nguoi
nguoi-test_nguoi
benh_nhan-test_nguoi
nguoi-test_nguoi
nguoi-test_nguoi
nguoi-test_nguoi
nguoi-test_nguoi
nguoi-test_nguoi
nguoi-test_nguoi
nguoi-test_nguoi
nguoi-test_nguoi
nguoi-test_nguoi
0.84
test_toi
toi-test_toi
toi-test_toi
toi-test_toi
khong-test_toi
toi-test_toi
toi-test_toi
benh_nhan-test_toi
mot-test_toi
toi-test_toi
toi-test_toi
toi-test_toi
toi-test_toi
nguoi-test_toi
toi-test_toi
toi-test_toi
toi-test_toi
toi-test_toi
benh_nhan-test_toi
benh_nhan-test_toi
toi-test_toi
toi-test_toi
toi-test_toi
toi-test_toi
khong-test_toi
toi-test_toi
0.72
test_khong
khong-test_khong
khong-test_khong
khong-test_khong
khong-test_khong
khong-test_khong
khong-test_khong
khong-test_khong
khong-test_khong
khong-test_khong
khong-te

In [ ]:
print(100*correct/count)